In [3]:
# 📦 Install if not already done
# !pip install langchain openai faiss-cpu tiktoken python-dotenv

# 1️⃣ Imports
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter

import os
from dotenv import load_dotenv

# 2️⃣ Load API keys
load_dotenv(".env")

# 3️⃣ Setup LLM
llm = ChatOpenAI(temperature=0)

# 4️⃣ Create Vector DB (Retriever)

with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=50)
texts = splitter.split_text(text_data)
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(texts, embedding)
retriever = vectorstore.as_retriever()


# 5️⃣ Create RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 6️⃣ Convert to LangChain Tool
qa_tool = Tool(
    name="LangChainRetriever",
    func=qa_chain.run,
    description="Use this to answer questions about LangChain framework, features, or its creator."
)

# 7️⃣ Setup Memory (to support conversational context)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 8️⃣ Initialize Agent with Tool + Memory
agent = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # ReAct agent
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 9️⃣ Ask Questions (RAG-Style)
print("1️⃣ First Question")
res1 = agent.run("Who had the glowing dial?")
print("Answer:", res1)

print("\n2️⃣ Follow-up")
res2 = agent.run("Which story mentions a magical painting?")
print("Answer:", res2)

print("\n3️⃣ Combined Reasoning")
res3 = agent.run("What was the message inside Bala’s lantern?")
print("Answer:", res3)


1️⃣ First Question


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LangChainRetriever
Action Input: Who had the glowing dial?
Observation: The broken pocket watch found by the young girl had the glowing dial.
Thought:Do I need to use a tool? No
AI: The broken pocket watch found by the young girl had the glowing dial.

> Finished chain.
Answer: The broken pocket watch found by the young girl had the glowing dial.

2️⃣ Follow-up


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LangChainRetriever
Action Input: Which story mentions a magical painting?
Observation: The story provided does not mention a magical painting; it focuses on a broken pocket watch with a strange glowing dial that reveals brief glimpses of the future.
Thought:Do I need to use a tool? No
AI: The story provided does not mention a magical painting; it focuses on a broken pocket watch with a strange glowing dial that reveals brief glimpses of the 